<a href="https://colab.research.google.com/github/etarazonav/650044-ABD-ULIMA/blob/main/Notebooks/ABB_SparkSQ4_Funciones_adicionales_fechas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Universidad_de_Lima_logo.png/220px-Universidad_de_Lima_logo.png"  width="120" />  Parte 4: Funciones Adicionales y Fechas
**Profesor:** Enver G. Tarazona Vargas <br>
**Curso:** Analítica con Big Data <br>
**FACULTAD DE INGENIERÍA - CARRERA DE INGENIERÍA DE SISTEMAS**<br>

In [ ]:
!pip install -q pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Carga de archivos
!wget -q https://raw.githubusercontent.com/etarazonav/650044-ABD-ULIMA/refs/heads/main/Datos/ventas.csv
!wget -q https://raw.githubusercontent.com/etarazonav/650044-ABD-ULIMA/refs/heads/main/Datos/covid/covid.csv

In [ ]:
df = spark.read.csv('ventas.csv', inferSchema=True, header=True)
df.show(4)

## 1.&nbsp;Funciones

Más información en: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

In [ ]:
# Ejemplos de funciones
from pyspark.sql.functions import avg, stddev, countDistinct, round

Conteo de elementos distintos

In [ ]:
df.select(countDistinct("Compania")).show()

Operaciones estadísticas como `stddev`, `avg`. Se puede utilizar `alias` para asignar un nombre más adecuado a la columna resultante y `round` para redondear.

In [ ]:
# Desviación estándar de las ventas
df.select(round(stddev("Ventas"), 2).alias("Desv ventas"),
          round(avg('Ventas'),2).alias('Media ventas')
          ).show()

Para el manejo de cadenas de caracteres se tiene algunas de las siguientes funciones

In [ ]:
from pyspark.sql.functions import concat, substring, length, trim, lower, upper, replace

df.select(concat("Compania", "Persona").alias("Concat"),
          substring("Compania", 1, 3).alias("Substring"),
          length("Compania").alias("Longitud"),
          lower("Compania").alias("minúscula"),
          upper("Compania").alias("mayúscula"),
).show()

Algunas funciones matemáticas aplicables por columna son las siguientes

In [ ]:
from pyspark.sql.functions import abs, round, ceil, floor, exp, log

df.select(abs("Ventas").alias("Valor_absoluto"),
          round("Ventas", 2).alias("Redondeo"),
          ceil("Ventas").alias("Ceil"),
          floor("Ventas").alias("Floor"),
          exp("Ventas").alias("Exp"),
          round(log("Ventas"),5).alias("Log")
          ).show(5)

La instrucción `when` asigna un valor condicionalmente

In [ ]:
from pyspark.sql.functions import when

df.select(round(df['Ventas']),
          when(df['Ventas'] >= 500, 'A').otherwise('B').alias('reemplazo'),
).show()

In [ ]:
from pyspark.sql.functions import when, col

df.withColumn('Compania_num',
              when(col('Compania') == 'Guguel', 0)
              .when(col('Compania') == 'Feisbuk', 1)
              .when(col('Compania') == 'Maikrosof', 2)
              .otherwise(3)
).show()

## 2.&nbsp;Fechas



In [ ]:
df = spark.read.csv('covid.csv', inferSchema=True, header=True)
df.show(4)

In [ ]:
df.select(df['fecha_ingreso_hosp'] ).show(5)

Se debe adecuar el formato a `DateTime`. Para realizar esta conversión se puede usar la función `to_date`. Para más información sobre los formatos ver: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

In [ ]:
from pyspark.sql.functions import to_date

# Creación de una columna llamada "ingreso" con formato de fecha "adecuado"
df = df.withColumn('ingreso', to_date(df['fecha_ingreso_hosp'], 'd/M/yyyy'))

df.select(['fecha_ingreso_hosp', 'ingreso']).show(5)

In [ ]:
from pyspark.sql.functions import year, month, dayofmonth, dayofyear, weekofyear

# Mostrar año, mes, día, semana, por separado
df.select(year(df['ingreso']).alias("año"),
          month(df['ingreso']).alias("mes"),
          dayofmonth(df['ingreso']).alias("día"),
          weekofyear(df['ingreso']).alias("semana"),
          dayofyear(df['ingreso']).alias("día del año")
          ).show(5)

### Ejemplo: agrupamiento con fechas

Se desea conocer qué años están presentes en el conjunto de datos. Se usará `distinct` para recuperar los valores distintos de años

In [ ]:
df.select(year(df['ingreso'])).distinct().show()

Se desea saber cuántos ingresos ocurrieron en cada año

In [ ]:
df.groupBy(year(df['ingreso'])).count().show()

Se desea analizar en qué mes ocurrió la mayor cantidad de ingresos, independientemente del año

In [ ]:
df.groupBy(month(df['ingreso']).alias('mes')) \
  .count() \
  .withColumnRenamed('count', 'total') \
  .orderBy('mes') \
  .show()

Se desea saber cuántos internamientos existen por año y por departamento de domicilio, y mostrar los 10 primeros ordenados de mayor a menor

In [ ]:
from pyspark.sql.functions import desc

df.groupBy(year(df['ingreso']), df['dep_domicilio']) \
  .count() \
  .orderBy(desc('count')) \
  .show(10)